In [1]:
# required libraries
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.linear_model import LogisticRegression, LinearRegression, BayesianRidge, \
    Lasso, SGDClassifier, TweedieRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonteamroster, playergamelog, teamgamelog
from nba_api.live.nba.endpoints import scoreboard, boxscore
from IPython.display import display, HTML

In [2]:
# endpoints: https://github.com/swar/nba_api/tree/master/docs/nba_api/stats/endpoints

header_dict = {
    'User-Agent': 'Mozilla/5.0',
    'x-nba-stats-origin': 'stats',
    'x-nba-stats-token': 'true',
    'Referer': 'https://stats.nba.com',
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
    'Host': 'stats.nba.com'
}


teams_dict = {'ATL': 0, 'BOS': 1, 'CHA': 2, 'CHI': 3, 'CLE': 4, 'DAL': 5, 'DEN': 6, 'DET': 7, 'GSW': 8, 'HOU': 9, 'IND': 10,
         'LAC': 11, 'LAL': 12, 'MEM': 13, 'MIA': 14, 'MIL': 15, 'MIN': 16, 'NOP': 17, 'NYK': 18, 'BKN': 19, 'OKC': 20,
         'ORL': 21, 'PHI': 22, 'PHX': 23, 'POR': 24, 'SAC': 25, 'SAS': 26, 'TOR': 27, 'UTA': 28, 'WAS': 29}


def get_playerID(name):
    all_players = players.get_players()

    for ind in range(len(all_players)):
        if all_players[ind]['full_name'].upper() == name.upper():
            return all_players[ind]['id']
    return ''

def get_teamID(team):
    all_teams = teams.get_teams()
    
    for item in all_teams:
        if item['abbreviation'] == team:
            return item['id']
    return ''

In [ ]:
all_teams = teams.get_teams()
data = []

for team in all_teams:
    # print(team)
    print('\n')

    team_id = team['id']
    abb = team['abbreviation']
    
    time.sleep(3)
    roster = commonteamroster.CommonTeamRoster(team_id=team_id, season='2022-23')
    roster = roster.get_dict()['resultSets'][0]['rowSet']

    d = {}
    for player in roster:        
        d = {'NAME': player[3], 'POS': player[7], 'TEAM': abb}
        print(d)
    
        data.append(d)

df = pd.DataFrame(data)
df.to_csv('team.csv', index=False)

In [19]:
# returns team matchups for the current season (2022-23)
team_abb = 'UTA'
team_id = get_teamID(team_abb)
matchups = teamgamelog.TeamGameLog(team_id=team_id)
matchups = matchups.get_dict()['resultSets'][0]['rowSet']
matchups_dates = []

data = []
for matchup in matchups:
    matchups_dates.append(matchup[2])    
    data.append(matchup[3][4:])

df = pd.DataFrame(columns=data[::-1])
display(HTML(df.to_html()))    

df.to_csv('matchups.csv', index=False)

,vs. DEN,@ MIN,@ NOP,@ HOU,vs. HOU,@ DEN,vs. MEM,vs. MEM,@ DAL,@ LAL,@ LAC,vs. LAL,@ ATL,@ WAS,@ PHI,vs. NYK,vs. PHX,@ POR,@ LAC,vs. DET,@ GSW,@ PHX,vs. CHI,vs. LAC,vs. IND,vs. POR,vs. GSW,vs. MIN,@ DEN,vs. NOP,vs. NOP,@ MIL,@ CLE,@ DET,vs. WAS,@ SAS,@ GSW,@ SAC,vs. MIA,vs. SAC,@ HOU,@ CHI,@ MEM,vs. CLE,vs. ORL,vs. PHI


In [20]:
# return each player's stats for the specified season (2022-23)
data = []
team_id = get_teamID(team_abb)
stat = 'PTS'
roster = commonteamroster.CommonTeamRoster(team_id=team_id, season='2022-23')
roster = roster.get_dict()['resultSets'][0]['rowSet']

for player in roster:    
    time.sleep(10)    
    d = {}
    
    print(player[3])
    player_id = player[14]
    game_log = playergamelog.PlayerGameLog(player_id=player_id)
    game_log = game_log.get_dict()['resultSets'][0]

    stats_names = game_log['headers']
    game_stats = game_log['rowSet']

    df = pd.DataFrame(game_stats, columns=stats_names)  
    
    matchup_dates_df = pd.DataFrame(matchups_dates, columns=['GAME_DATE'])
    df1 = df[['GAME_DATE', 'MATCHUP', stat]]    
    
    df = matchup_dates_df.merge(df1, on=['GAME_DATE'], how='left', indicator='Match')
    
    d = df[stat][::-1].to_dict()        
    data.append(d)


df = pd.DataFrame(data)
# display(HTML(df.to_html()))
df.to_csv('player_stats.csv', index=False)

Talen Horton-Tucker
Jordan Clarkson
Collin Sexton
Malik Beasley
Nickeil Alexander-Walker
Jarred Vanderbilt
Leandro Bolmaro
Mike Conley
Simone Fontecchio
Udoka Azubuike
Rudy Gay
Lauri Markkanen
Walker Kessler
Micah Potter
Ochai Agbaji
Johnny Juzang
Kelly Olynyk
